# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test


## Define dictionary

In [15]:
## Read ref_launch.yaml
# main_code_dir=config_dict[facility]['main_code_dir']
# dict_pars=config_dict[facility][img_size][run_type]
dict_pars={'nodes':10, 'queue':'regular','bins':'unenven','cores':64,'time':'0:30:00',
           'job_name':'3ptfnc',
            'suffix':'inf_20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50_step_42-24720_sigma0.5','num_l':8,'img_slice':64,
                 }

dict_pars['staging_loc']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/'
dict_pars['code_dir']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test'
dict_pars['cores']=dict_pars['nodes']*64

In [16]:
### 3dgan 64cube
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-124_step-19390.npy'
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-118_step-18560.npy'


In [17]:
### 3dcgan 64cube
sigma=0.5
#dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma)

dict_pars['fname']='20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/images/inference_label-{0}_epoch-42_step-24720.npy'.format(sigma)

In [18]:
# %%bash -s "$op_fldr"
# ls $1

In [19]:
dict_pars

{'nodes': 10,
 'queue': 'regular',
 'bins': 'unenven',
 'cores': 640,
 'time': '0:30:00',
 'job_name': '3ptfnc',
 'suffix': 'inf_20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50_step_42-24720_sigma0.5',
 'num_l': 8,
 'img_slice': 64,
 'staging_loc': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/',
 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test',
 'fname': '20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/images/inference_label-0.5_epoch-42_step-24720.npy'}

In [20]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir={code_dir}
fname={fname}

srun -n {cores} python $code_dir/3pt_fcn_simple.py -n {num_l} --img_slice {img_slice} -idx $I -f $fname -sfx {suffix}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [21]:
fname='batch_3ptcn.sh'
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/batch_3ptcn.sh


In [22]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [23]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=10
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=0:30:00
#SBATCH --job-name=3ptfnc

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test
fname=20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/images/inference_label-0.5_epoch-42_step-24720.npy

srun -n 640 python $code_dir/3pt_fcn_simple.py -n 8 --img_slice 64 -idx $I -f $fname -sfx inf_20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50_step_42-24720_sigma0.5

echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [24]:
lst=np.arange(128)
# lst=[0,3,6,8,19]
lst2=[str(i) for i in lst]
strg=' '.join(lst2)
print(strg)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127


In [25]:
%%bash -s "$filename" "$strg" ## Use python variable in bash
# for i in $2; do echo $i; echo $1; done
for i in $2; do sbatch $1 $i; done

Submitted batch job 43642957
Submitted batch job 43642958
Submitted batch job 43642959
Submitted batch job 43642960
Submitted batch job 43642961
Submitted batch job 43642962
Submitted batch job 43642963
Submitted batch job 43642964
Submitted batch job 43642965
Submitted batch job 43642966
Submitted batch job 43642967
Submitted batch job 43642968
Submitted batch job 43642969
Submitted batch job 43642970
Submitted batch job 43642971
Submitted batch job 43642972
Submitted batch job 43642973
Submitted batch job 43642974
Submitted batch job 43642975
Submitted batch job 43642976
Submitted batch job 43642977
Submitted batch job 43642978
Submitted batch job 43642979
Submitted batch job 43642980
Submitted batch job 43642981
Submitted batch job 43642982
Submitted batch job 43642983
Submitted batch job 43642984
Submitted batch job 43642985
Submitted batch job 43642986
Submitted batch job 43642987
Submitted batch job 43642988
Submitted batch job 43642989
Submitted batch job 43642990
Submitted batc

In [4]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/images/inference_label-0.5_epoch-36_step-21550.npy'
np.load(fname,mmap_mode='r').shape

(128, 1, 64, 64, 64)